In [7]:
import os
import numpy as np
import tensorflow as tf
from sklearn.model_selection import KFold, train_test_split # Importar train_test_split
from tensorflow.keras.models import load_model # Importar para carregar o modelo da fase 1
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, EarlyStopping

from core import criar_modelo, ParImageGenerator, create_tensorboard_callback

In [8]:
# --- Configurações Gerais ---
# Desative se não estiver depurando para melhor performance em produção
# tf.config.run_functions_eagerly(True)

# Configuracao do TensorFlow para usar a GPU, se disponível
# Verifica se a variável de ambiente CUDA_VISIBLE_DEVICES está definida
cuda_visible_devices = os.getenv('CUDA_VISIBLE_DEVICES', None)
if cuda_visible_devices is not None:
    print(f"🔧 Variável de ambiente CUDA_VISIBLE_DEVICES definida: {cuda_visible_devices}")
try:
    gpus = tf.config.list_physical_devices('GPU')
    if gpus:
        # Configura a memória da GPU para crescer dinamicamente, evitando erros de "Out of Memory" (OOM)
        # ao tentar alocar toda a memória da GPU de uma vez.
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print(f"✅ GPU(s) detectada(s) e configurada(s) para uso dinâmico de memória: {gpus}")
        print("TensorFlow utilizará a(s) GPU(s) disponível(is).")
    else:
        print("❌ Nenhuma GPU compatível detectada. O TensorFlow será executado na CPU.")
        # Opcional: Se você quisesse forçar o uso da CPU mesmo com GPU, poderia fazer:
        # tf.config.set_visible_devices([], 'GPU') # Esconde todas as GPUs do TensorFlow
        print("Continuando a execução na CPU.")
except RuntimeError as e:
    # Captura erros que podem ocorrer se a GPU não estiver configurada corretamente
    print(f"❌ Erro ao configurar GPU: {e}")
    print("O TensorFlow será executado na CPU devido ao erro na configuração da GPU.")
    # Opcional: Para garantir a CPU após um erro, embora TensorFlow faça isso por padrão
    # tf.config.set_visible_devices([], 'GPU')

❌ Nenhuma GPU compatível detectada. O TensorFlow será executado na CPU.
Continuando a execução na CPU.


In [9]:
# Parâmetros para a divisão Treino / Validação / Teste (FIXOS)
TRAIN_SIZE_RATIO = 0.60 # 60% para o conjunto de treino
VAL_SIZE_RATIO = 0.20   # 20% para o conjunto de validação
TEST_SIZE_RATIO = 0.20  # 20% para o conjunto de teste

GLOBAL_RANDOM_STATE = 42 # Para reprodutibilidade das divisões

In [10]:
# Caminhos globais
MODEL_DIR = '../model'
RESULTS_DIR = '../results'
LOGS_DIR = '../logs'
CV_LOG_PATH = os.path.join(RESULTS_DIR, 'cv_info.txt')
FINAL_TEST_LOG_PATH = os.path.join(RESULTS_DIR, 'final_test_evaluation.txt') # Novo log para teste final
IMAGE_TRAINING_DIR = '../image/treinamento'
BATCH_SIZE = 8
N_SPLITS = 5 # N_SPLITS para o K-Fold (operando apenas no conjunto de treino)
KFOLD_RANDOM_STATE = 42 # Random state para o KFold

In [11]:
# --- Configurações das Fases de Treinamento (Hardcoded no script) ---
# FASE 1 Treinamento Inicial
PHASE1_EPOCHS = 1 # Ajuste para um valor maior (ex: 15-20) para treinamento real
PHASE1_LEARNING_RATE = 0.0001
PHASE1_ES_PATIENCE = 7
PHASE1_CHECKPOINT_FORMAT = f'{MODEL_DIR}/modelo_ph1_fold{{fold}}.h5'
PHASE1_HISTORY_LOG_FORMAT = f'{RESULTS_DIR}/historico_fase1_fold{{fold}}.csv'
PHASE1_TENSORBOARD_SUFFIX_FORMAT = 'fold_{fold}/fase_1'

# FASE 2 Fine-Tuning Aprofundado
PHASE2_EPOCHS = 1 # Ajuste para um valor maior (ex: 10-30) para treinamento real
PHASE2_LEARNING_RATE = 1e-5 # LR mais baixo para fine-tuning aprofundado
PHASE2_ES_PATIENCE = 7
PHASE2_CHECKPOINT_FORMAT = f'{MODEL_DIR}/final_tuned_fold{{fold}}.h5'
PHASE2_HISTORY_LOG_FORMAT = f'{RESULTS_DIR}/historico_fase2_fold{{fold}}.csv'
PHASE2_TENSORBOARD_SUFFIX_FORMAT = 'fold_{fold}/fase_2'
PHASE2_FREEZE_LAYERS = 100 # Camadas a congelar na base na Fase 2

In [12]:
# --- Criação de Diretórios ---
os.makedirs(MODEL_DIR, exist_ok=True)
os.makedirs(RESULTS_DIR, exist_ok=True)
os.makedirs(LOGS_DIR, exist_ok=True)

In [18]:
# --- FUNÇÃO AUXILIAR LOCAL PARA EXECUTAR UMA FASE DE TREINAMENTO ---
def _run_training_phase(
    model: tf.keras.Model,
    train_gen,
    val_gen,
    fold: int,
    phase_name: str,
    epochs: int,
    learning_rate: float,
    checkpoint_filepath: str,
    history_log_filepath: str,
    tensorboard_log_suffix: str,
    patience: int
) -> tf.keras.Model:
    """
    Executa uma única fase de treinamento para o modelo, configurando
    o compilador e os callbacks com base nas configurações da fase.

    Args:
        model (tf.keras.Model): O modelo a ser treinado.
        train_gen: Gerador de dados de treinamento.
        val_gen: Gerador de dados de validação.
        fold (int): Número do fold atual.
        phase_name (str): Nome descritivo da fase (e.g., 'Fase 1', 'Fase 2').
        epochs (int): Número de épocas.
        learning_rate (float): Taxa de aprendizado.
        checkpoint_filepath (str): Caminho para salvar o checkpoint do modelo.
        history_log_filepath (str): Caminho para o log CSV.
        tensorboard_log_suffix (str): Sufixo para o diretório de logs do TensorBoard.
        patience (int): Paciência para EarlyStopping.

    Returns:
        tf.keras.Model: O modelo após o treinamento da fase, com os melhores pesos restaurados pelo EarlyStopping.
    """
    print(f"--- Iniciando {phase_name} para Fold {fold} ---")

    # Compila o modelo com o learning rate específico da fase
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    # Configura os Callbacks para esta fase
    checkpoint = ModelCheckpoint(
        filepath=checkpoint_filepath,
        monitor='val_loss',
        save_best_only=True,
        verbose=1
    )
    logger = CSVLogger(history_log_filepath, append=False)
    early_stopping = EarlyStopping(
        monitor='val_loss',
        patience=patience,
        restore_best_weights=True,
        verbose=1
    )
    tensorboard_callback = create_tensorboard_callback(fold_name=tensorboard_log_suffix)

    # Executa o treinamento
    history = model.fit(
        train_gen,
        validation_data=val_gen,
        epochs=epochs,
        # steps_per_epoch=len(train_gen),
        steps_per_epoch=1,
        callbacks=[checkpoint, logger, early_stopping, tensorboard_callback],
        verbose=1
    )

    print(f"--- {phase_name} para Fold {fold} concluída ---")

    return model

In [19]:
# ============================
# 🔁 Carregar todos os dados
# ============================
print("Carregando todos os dados...")
gen_temporario = ParImageGenerator(IMAGE_TRAINING_DIR, batch_size=1, augmentacao=False)
dados_completos = list(zip(gen_temporario.imagens, gen_temporario.labels))
print(f"Total de pares de imagens carregados: {len(dados_completos)}")

Carregando todos os dados...
Total de pares de imagens carregados: 569


In [20]:
# ============================
# 🔁 Divisão Treino (60%) / Validação (20%) / Teste (20%) - FIXOS
# ============================
print(f"Dividindo dados: {TRAIN_SIZE_RATIO*100:.0f}% Treino, {VAL_SIZE_RATIO*100:.0f}% Validação, {TEST_SIZE_RATIO*100:.0f}% Teste.")

# 1. Separar o conjunto de Teste (20% do total)
dados_treino_val_temp, dados_teste_fixo = train_test_split(
    dados_completos,
    test_size=TEST_SIZE_RATIO,
    random_state=GLOBAL_RANDOM_STATE,
    stratify=[label for _, label in dados_completos] # Garante proporção de classes
)

# 2. Separar o conjunto de Validação Fixo (20% do total, ou 25% do restante de treino_val_temp)
dados_treino_para_kfold, dados_validacao_fixo = train_test_split(
    dados_treino_val_temp,
    test_size=(VAL_SIZE_RATIO / (TRAIN_SIZE_RATIO + VAL_SIZE_RATIO)), # Calcula a proporção para o segundo split (0.20 / 0.80 = 0.25)
    random_state=GLOBAL_RANDOM_STATE,
    stratify=[label for _, label in dados_treino_val_temp]
)

print(f"Pares para Treino (Fixo para K-Fold): {len(dados_treino_para_kfold)} (~{TRAIN_SIZE_RATIO*100:.0f}%)")
print(f"Pares para Validação (Fixo): {len(dados_validacao_fixo)} (~{VAL_SIZE_RATIO*100:.0f}%)")
print(f"Pares para Teste (Fixo): {len(dados_teste_fixo)} (~{TEST_SIZE_RATIO*100:.0f}%)")


Dividindo dados: 60% Treino, 20% Validação, 20% Teste.
Pares para Treino (Fixo para K-Fold): 341 (~60%)
Pares para Validação (Fixo): 114 (~20%)
Pares para Teste (Fixo): 114 (~20%)


In [21]:
# ============================
# 🔁 K-Fold Cross-Validation NO CONJUNTO DE TREINO FIXO (60%)
# ============================
kf = KFold(n_splits=N_SPLITS, shuffle=True, random_state=KFOLD_RANDOM_STATE)

fold = 1
# Acúmulo de métricas para a média do K-Fold (agora, a validação é sempre no conjunto fixo)
accuracies_cv = []
losses_cv = []

with open(CV_LOG_PATH, 'w') as log_file_cv:
    log_file_cv.write(f"--- Log de Cross-Validation ({N_SPLITS} Folds) ---\n")
    log_file_cv.write(f"K-Fold operando no conjunto de Treino Fixo ({len(dados_treino_para_kfold)} pares).\n")
    log_file_cv.write(f"Conjunto de Validação Fixo para todos os folds: {len(dados_validacao_fixo)} pares.\n\n")


    # Criar o gerador de validação FIXO para TODOS os folds
    val_gen_fixo = ParImageGenerator(dados=dados_validacao_fixo, batch_size=BATCH_SIZE, augmentacao=False)


    for train_idx_interno, val_idx_interno in kf.split(dados_treino_para_kfold): # K-Fold APENAS no conjunto de treino fixo
        log_file_cv.write(f"[Fold {fold}]\n")
        log_file_cv.write(f"Índices de Treino Interno do Fold: {len(train_idx_interno)} pares\n")
        log_file_cv.write(f"Índices de Validação Interna do Fold (não usado aqui): {len(val_idx_interno)} pares\n\n") # Este val_idx_interno NÃO será usado

        # Os dados de treinamento para o 'fit' virão apenas do fold de treino
        train_data_fold = [dados_treino_para_kfold[i] for i in train_idx_interno]
        train_gen_fold = ParImageGenerator(dados=train_data_fold, batch_size=BATCH_SIZE, augmentacao=True)


        # --- FASE 1: Fine-tuning Inicial ---
        print(f"\n🌀 Treinando Fold {fold} - Fase 1 (Fine-tuning inicial)...")
        modelo_fase1 = criar_modelo() # Cria o modelo para a Fase 1

        modelo_fase1_treinado = _run_training_phase(
            model=modelo_fase1,
            train_gen=train_gen_fold, # Usar o gerador de TREINO do fold
            val_gen=val_gen_fixo,   # USAR O GERADOR DE VALIDAÇÃO FIXO
            fold=fold,
            phase_name="Fase 1 (Fine-tuning inicial)",
            epochs=PHASE1_EPOCHS,
            learning_rate=PHASE1_LEARNING_RATE,
            checkpoint_filepath=PHASE1_CHECKPOINT_FORMAT.format(fold=fold),
            history_log_filepath=PHASE1_HISTORY_LOG_FORMAT.format(fold=fold),
            tensorboard_log_suffix=PHASE1_TENSORBOARD_SUFFIX_FORMAT.format(fold=fold),
            patience=PHASE1_ES_PATIENCE
        )

        # Carregar o melhor modelo da Fase 1 para garantir a continuidade correta
        model_path_ph1 = PHASE1_CHECKPOINT_FORMAT.format(fold=fold)
        try:
            modelo_para_fase2 = load_model(model_path_ph1)
            print(f"✅ Modelo da Fase 1 carregado para a Fase 2.")
        except Exception as e:
            print(f"AVISO: Não foi possível carregar o melhor modelo da Fase 1 para Fold {fold}. Erro: {e}")
            print("AVISO: A Fase 2 não poderá iniciar. Pulando para o próximo fold.")
            continue


        # --- FASE 2: Fine-tuning Aprofundado ---
        print(f"\n🔧 Treinando Fold {fold} - Fase 2 (Fine-tuning aprofundado)...")

        # Ajusta as camadas treináveis para a Fase 2 (congelando as 100 primeiras)
        # O modelo carregado (modelo_para_fase2) é uma nova instância do modelo original,
        # então suas camadas treináveis precisam ser reconfiguradas.
        if hasattr(modelo_para_fase2.layers[0], 'layers'):
            base_model_ph2 = modelo_para_fase2.layers[0]
            base_model_ph2.trainable = True # Garante que a base está treinável antes de congelar partes
            for layer in base_model_ph2.layers[:PHASE2_FREEZE_LAYERS]:
                layer.trainable = False
        else:
            print(f"AVISO: A primeira camada do modelo do Fold {fold} não parece ser a base ResNet50. Fine-tuning da Fase 2 pode não funcionar como esperado.")


        modelo_fase2_treinado = _run_training_phase(
            model=modelo_para_fase2,
            train_gen=train_gen_fold, # Usar o gerador de TREINO do fold
            val_gen=val_gen_fixo,   # USAR O GERADOR DE VALIDAÇÃO FIXO
            fold=fold,
            phase_name="Fase 2 (Fine-tuning aprofundado)",
            epochs=PHASE2_EPOCHS,
            learning_rate=PHASE2_LEARNING_RATE,
            checkpoint_filepath=PHASE2_CHECKPOINT_FORMAT.format(fold=fold),
            history_log_filepath=PHASE2_HISTORY_LOG_FORMAT.format(fold=fold),
            tensorboard_log_suffix=PHASE2_TENSORBOARD_SUFFIX_FORMAT.format(fold=fold),
            patience=PHASE2_ES_PATIENCE
        )

        # Avaliação final do modelo da Fase 2 para este fold
        # AVALIADO NO CONJUNTO DE VALIDAÇÃO FIXO
        loss, accuracy = modelo_fase2_treinado.evaluate(val_gen_fixo, verbose=1)
        print(f"📊 [Fold {fold}] Loss no Validação Fixo: {loss:.4f}, Accuracy no Validação Fixo: {accuracy:.4f}")

        log_file_cv.write(f"Resultado Final Fold {fold} (no Val. Fixo): Loss={loss:.4f}, Accuracy={accuracy:.4f}\n\n")

        accuracies_cv.append(accuracy)
        losses_cv.append(loss)
        fold += 1



🌀 Treinando Fold 1 - Fase 1 (Fine-tuning inicial)...
--- Iniciando Fase 1 (Fine-tuning inicial) para Fold 1 ---
✅ Logs do TensorBoard serão salvos em: c:\cod\python\AI\logs\fold_1\fase_1


c:\cod\python\AI\venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64s/step - accuracy: 1.0000 - loss: 0.4259
Epoch 1: val_loss improved from inf to 0.51185, saving model to ../model/modelo_ph1_fold1.h5


1/1 ━━━━━━━━━━━━━━━━━━━━ 108s 108s/step - accuracy: 1.0000 - loss: 0.4259 - val_accuracy: 0.8482 - val_loss: 0.5119
Restoring model weights from the end of the best epoch: 1.
--- Fase 1 (Fine-tuning inicial) para Fold 1 concluída ---


✅ Modelo da Fase 1 carregado para a Fase 2.

🔧 Treinando Fold 1 - Fase 2 (Fine-tuning aprofundado)...
AVISO: A primeira camada do modelo do Fold 1 não parece ser a base ResNet50. Fine-tuning da Fase 2 pode não funcionar como esperado.
--- Iniciando Fase 2 (Fine-tuning aprofundado) para Fold 1 ---
✅ Logs do TensorBoard serão salvos em: c:\cod\python\AI\logs\fold_1\fase_2
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68s/step - accuracy: 0.8750 - loss: 0.4497
Epoch 1: val_loss improved from inf to 0.50198, saving model to ../model/final_tuned_fold1.h5


...

✅ Modelo da Fase 1 carregado para a Fase 2.

🔧 Treinando Fold 4 - Fase 2 (Fine-tuning aprofundado)...
AVISO: A primeira camada do modelo do Fold 4 não parece ser a base ResNet50. Fine-tuning da Fase 2 pode não funcionar como esperado.
--- Iniciando Fase 2 (Fine-tuning aprofundado) para Fold 4 ---
✅ Logs do TensorBoard serão salvos em: c:\cod\python\AI\logs\fold_4\fase_2
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69s/step - accuracy: 0.7500 - loss: 0.4532
Epoch 1: val_loss improved from inf to 0.43120, saving model to ../model/final_tuned_fold4.h5


1/1 ━━━━━━━━━━━━━━━━━━━━ 106s 106s/step - accuracy: 0.7500 - loss: 0.4532 - val_accuracy: 0.8482 - val_loss: 0.4312
Restoring model weights from the end of the best epoch: 1.
--- Fase 2 (Fine-tuning aprofundado) para Fold 4 concluída ---
14/14 ━━━━━━━━━━━━━━━━━━━━ 24s 2s/step - accuracy: 0.8041 - loss: 0.5197
📊 [Fold 4] Loss no Validação Fixo: 0.4133, Accuracy no Validação Fixo: 0.8571

🌀 Treinando Fold 5 - Fase 1 (Fine-tuning inicial)...
--- Iniciando Fase 1 (Fine-tuning inicial) para Fold 5 ---
✅ Logs do TensorBoard serão salvos em: c:\cod\python\AI\logs\fold_5\fase_1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66s/step - accuracy: 0.6250 - loss: 0.6475
Epoch 1: val_loss improved from inf to 0.93878, saving model to ../model/modelo_ph1_fold5.h5


1/1 ━━━━━━━━━━━━━━━━━━━━ 102s 102s/step - accuracy: 0.6250 - loss: 0.6475 - val_accuracy: 0.1518 - val_loss: 0.9388
Restoring model weights from the end of the best epoch: 1.
--- Fase 1 (Fine-tuning inicial) para Fold 5 concluída ---


✅ Modelo da Fase 1 carregado para a Fase 2.

🔧 Treinando Fold 5 - Fase 2 (Fine-tuning aprofundado)...
AVISO: A primeira camada do modelo do Fold 5 não parece ser a base ResNet50. Fine-tuning da Fase 2 pode não funcionar como esperado.
--- Iniciando Fase 2 (Fine-tuning aprofundado) para Fold 5 ---
✅ Logs do TensorBoard serão salvos em: c:\cod\python\AI\logs\fold_5\fase_2
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67s/step - accuracy: 0.8750 - loss: 0.3096
Epoch 1: val_loss improved from inf to 0.93255, saving model to ../model/final_tuned_fold5.h5


1/1 ━━━━━━━━━━━━━━━━━━━━ 104s 104s/step - accuracy: 0.8750 - loss: 0.3096 - val_accuracy: 0.1518 - val_loss: 0.9325
Restoring model weights from the end of the best epoch: 1.
--- Fase 2 (Fine-tuning aprofundado) para Fold 5 concluída ---
14/14 ━━━━━━━━━━━━━━━━━━━━ 23s 2s/step - accuracy: 0.1257 - loss: 0.9403
📊 [Fold 5] Loss no Validação Fixo: 0.9327, Accuracy no Validação Fixo: 0.1518


In [22]:
# ============================
# 📈 Resultados de Cross-Validation (no Conjunto de Validação Fixo)
# ============================
media_acc_cv = np.mean(accuracies_cv)
media_loss_cv = np.mean(losses_cv)

print(f"\n✅ Cross-validation no conjunto de treino finalizada!")
print(f"📉 Média da Loss (Validação Fixo em CV): {media_loss_cv:.4f}")
print(f"✅ Média da Accuracy (Validação Fixo em CV): {media_acc_cv:.4f}")


✅ Cross-validation no conjunto de treino finalizada!
📉 Média da Loss (Validação Fixo em CV): 0.6808
✅ Média da Accuracy (Validação Fixo em CV): 0.5714


In [27]:
with open(CV_LOG_PATH, 'a') as log_file_cv:
    log_file_cv.write("=== Resultado Final da Cross-Validation (no Validação Fixo) ===\n")
    log_file_cv.write(f"Média da Loss (Validação Fixo em CV): {media_loss_cv:.4f}\n")
    log_file_cv.write(f"Média da Accuracy (Validação Fixo em CV): {media_acc_cv:.4f}\n")

In [30]:
# ============================
# 🧪 Avaliação Final no Conjunto de Teste Intocado (20% Fixo)
# ============================
print(f"\n🚀 Avaliando o modelo final médio no Conjunto de Teste ({len(dados_teste_fixo)} pares)...")

MODELO_FINAL_MEDIA_PATH = os.path.join(MODEL_DIR, 'modelo_final_media.h5')

try:
    final_model_for_test = load_model(MODELO_FINAL_MEDIA_PATH)
    test_gen_fixo = ParImageGenerator(dados=dados_teste_fixo, batch_size=BATCH_SIZE, augmentacao=False) # Sem aumento para teste

    test_loss, test_accuracy = final_model_for_test.evaluate(test_gen_fixo, verbose=1)

    print(f"\n🎉 Resultado Final no Conjunto de Teste:")
    print(f"   Loss no Teste: {test_loss:.4f}")
    print(f"   Accuracy no Teste: {test_accuracy:.4f}")

    with open(FINAL_TEST_LOG_PATH, 'w') as f_test_log:
        f_test_log.write(f"=== Avaliação Final no Conjunto de Teste ===\n")
        f_test_log.write(f"Total de pares de teste: {len(dados_teste_fixo)}\n")
        f_test_log.write(f"Loss no Teste: {test_loss:.4f}\n")
        f_test_log.write(f"Accuracy no Teste: {test_accuracy:.4f}\n")
    print(f"Resultados do teste final salvos em: {FINAL_TEST_LOG_PATH}")

except FileNotFoundError:
    print(f"❌ ERRO: O modelo final médio '{MODELO_FINAL_MEDIA_PATH}' não foi encontrado.")
    print("Por favor, execute 'python src/modelo_final.py' primeiro para criar o modelo final.")
except Exception as e:
    print(f"❌ ERRO ao avaliar o modelo final no conjunto de teste: {e}")


🚀 Avaliando o modelo final médio no Conjunto de Teste (114 pares)...


14/14 ━━━━━━━━━━━━━━━━━━━━ 13s 673ms/step - accuracy: 0.8265 - loss: 0.5742

🎉 Resultado Final no Conjunto de Teste:
   Loss no Teste: 0.5606
   Accuracy no Teste: 0.8571
Resultados do teste final salvos em: ../results\final_test_evaluation.txt
